In [2]:
%pip install crewai==0.55.2 crewai_tools langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [3]:
#%pip install composio-core

In [4]:
%pip install composio_crewai

Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
from composio import App
from crewai_tools import ComposioTool
from composio import Action
"""

SyntaxError: incomplete input (2818087860.py, line 1)

In [5]:
from crewai import Agent, Task, Crew,Process
from langchain_groq import ChatGroq
import os

In [6]:
from composio_crewai import ComposioToolSet, App,Action

In [7]:
os.environ["COMPOSIO_API_KEY"] = "nadbnqfuprr4r8zgmqxaoh"


In [8]:
os.environ["GROQ_API_KEY"] = "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"
llm = ChatGroq(
    model_name="gemma2-9b-it",
    # other params...
)

In [13]:
toolset = ComposioToolSet(entity_id="joseph")

entity = toolset.get_entity()

request = entity.initiate_connection(App.GMAIL)

print(
    f"Open this URL in your browser: {request.redirectUrl}"
)

2024-09-21 07:42:20,513 - 140269761352320 - logging.py-logging:85 - INFO: Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


Open this URL in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=967786027051-8p515bt7ijf94c2bf4a5lev5jr6r7oc1.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fbackend.composio.dev%2Fapi%2Fv1%2Fauth-apps%2Fadd&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&response_type=code&access_type=offline&prompt=consent&state=production_eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJjb25uZWN0aW9uSWQiOiJiYjMyMWRjYy00OGRmLTRjM2UtOTg0Mi03MzU3MzViZmM2MzAiLCJpbnRlZ3JhdGlvbklkIjoiNjFiNDY1ZWQtYThiYy00MTg3LWFhMGMtYWRlNjIxMGEzOGVmIiwiYXBwTmFtZSI6ImdtYWlsIiwiY2xpZW50SW5mbyI6eyJjbGllbnRJZCI6ImY4MTI4ZDU4LWNhN2MtNGE4YS1iNTQ2LTkyYTEyNDJmOTEzNCJ9LCJpYXQiOjE3MjY5MDQ1NDB9.w0apMAchTNs_dCho0EMO714FJxELUW0Y6Wc6qV2ShK8&code_challenge=F-fLUVn_QvmcWsVsAOzTJVxRTiUTPQbulahomQWU-Fw&code_challenge_method=S256


In [9]:
composio_toolset = ComposioToolSet()
#tools = composio_toolset.get_tools(apps=[App.GMAIL])
tools = composio_toolset.get_tools(actions=[Action.GMAIL_SEND_EMAIL])

2024-09-21 08:52:32,102 - 140197120762496 - logging.py-logging:85 - INFO: Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this


In [11]:
#tools = [ComposioTool.from_action(action=Action.GMAIL_SEND_EMAIL)]
#tools = ComposioTool.from_app(App.GMAIL, use_case="Send Email")

In [10]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic} in 'https://substack.com/'."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "You have to prepare a detailed "
              "outline and the relevant topics and sub-topics that has to be a part of the"
              "blogpost."
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
 verbose=True
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic} in 'https://substack.com/'. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization 'https://substack.com/'. ",
    backstory="You are an editor who receives a blog post "
              "from the agent Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [11]:
emailsender = Agent(
    role="Email Sender",
    goal="Send the blog post genertated by agent Editor to Email:{Receiver}",
    backstory="You're working on sending a blog article "
              "to {Receiver}."
              "Email Subject should be 'Blog Article genertated by AI' "
              "Email Body should be the output result of agent Editor,Ensure the integrity of the content being sent.",
    llm=llm,
    tools = tools,
    allow_delegation=False,
 verbose=True
)

In [12]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [13]:
emailsend = Task(
    description=("send the edited blog post to {Receiver} "
                 ),
    expected_output="the response status of sent email",
    agent=emailsender
)

In [14]:
crew = Crew(
    agents=[planner, writer, editor, emailsender],
    tasks=[plan, write, edit, emailsend],
    process=Process.sequential,
    verbose=True
)

#inputs = {"topic":"Comparative study of LangGraph, Autogen and Crewai for building multi-agent system."}
inputs = {"topic":"the recommendation about the top 5 cafe in paris.","Receiver":"josephchan7939@gmail.com"}
result = crew.kickoff(inputs=inputs)



 [2024-09-21 08:52:56][DEBUG]: == Working Agent: Content Planner
 [2024-09-21 08:52:56][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on the recommendation about the top 5 cafe in paris..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...


Thought: I now can give a great answer 

Final Answer: 

##  Parisian Coffee Culture: Uncovering the Top 5 Cafés You Can't Miss 

**Content Plan Document**

**1. Target Audience:**

* **Primary:**  Travel enthusiasts, foodies, and individuals interested in experiencing authentic Parisian culture. 
* **Secondary:**  Digital nomads, remote workers, and students seeking charming and inspiring workspaces.
* **Interests:**  Exploring new cultures, savoring delicious food and coffee, photography, unique experiences, Parisian history and architecture.
* **Pain Points:**  Finding hidden gems, navigating language barriers, wanting to experience Paris beyond tourist traps.

**2. Latest Trends & Key Players:**

* **Focus on Specialty Coffee:**  Paris is seeing a rise in specialty coffee shops offering single-origin beans, pour-overs, and expertly crafted espresso drinks.
* **Sustainable Practices:**  Many cafés are prioritizing ethical sourcing, eco-friendly packaging, and minimizing waste.
* **I

2024-09-21 08:53:02,485 - 140197120762496 - logging.py-logging:85 - INFO: Executing `GMAIL_SEND_EMAIL` with params={'user_id': 'me', 'recipient_email': 'josephchan7939@gmail.com', 'subject': 'Blog Article generated by AI', 'body': "## A Parisian Awakening: 5 Cafés You Won't Want to Miss\n\nParisian cafés aren't just places to gr...


Thought: I have all the necessary information to send the email.
Action: GMAIL_SEND_EMAIL
Action Input: {'recipient_email': 'josephchan7939@gmail.com', 'subject': 'Blog Article generated by AI', 'body': '## A Parisian Awakening: 5 Cafés You Won\'t Want to Miss\n\nParisian cafés aren\'t just places to grab a caffeine fix; they\'re portals to a world of timeless charm, intellectual discourse, and exquisite coffee rituals. The air hums with a blend of warm, roasted aromas and lively conversation, creating an atmosphere both cozy and invigorating.  Whether you\'re seeking a literary haven, a glimpse into history, or simply a moment of peaceful contemplation, Paris\'s café culture has something to offer everyone. \n\nHere are five cafés that encapsulate the essence of Parisian charm, each with its unique character and allure:\n\n**1. Café de Flore: Where Art and History Collide**\n\nNestled in the heart of Saint-Germain-des-Prés, Café de Flore is more than just a café; it's a living legend.

2024-09-21 08:53:03,233 - 140197120762496 - logging.py-logging:85 - INFO: Got response={'successfull': True, 'data': {'response_data': {'id': '19213c8d958a34d3', 'threadId': '19213c8d958a34d3', 'labelIds': ['SENT']}}, 'error': None} from action=<composio.client.enums._action.Action object at 0x7f81dd3f3610> with params={'user_id...


 

{'successfull': True, 'data': {'response_data': {'id': '19213c8d958a34d3', 'threadId': '19213c8d958a34d3', 'labelIds': ['SENT']}}, 'error': None}

Thought: I now can give a great answer
Final Answer: {'successfull': True, 'data': {'response_data': {'id': '19213c8d958a34d3', 'threadId': '19213c8d958a34d3', 'labelIds': ['SENT']}}, 'error': None} 


> Finished chain.
 [2024-09-21 08:53:03][DEBUG]: == [Email Sender] Task output: {'successfull': True, 'data': {'response_data': {'id': '19213c8d958a34d3', 'threadId': '19213c8d958a34d3', 'labelIds': ['SENT']}}, 'error': None}




In [21]:
from IPython.display import Markdown,display
display(Markdown(result.raw))

## Beyond the Croissants: 5 Parisian Cafés to Experience

Paris. City of Lights. Romance. Iconic landmarks. And, of course, its legendary cafés. Stepping into a Parisian café is more than just grabbing a coffee; it's about immersing yourself in the city's soul, savoring the art de vivre, and soaking up the atmosphere.  But with countless cafés lining every corner, choosing where to sip your espresso or indulge in a flaky croissant can be overwhelming. Fear not, fellow caffeine lovers and culture seekers!  I've curated a list of five exceptional Parisian cafés that go beyond the ordinary, each offering a unique blend of ambiance, quality, and Parisian charm.

### 1. Coutume Café: Where Coffee is an Art Form

Tucked away in the heart of the Marais district, Coutume Café is a haven for coffee enthusiasts. It's not just a café; it's a temple dedicated to the art of brewing. Coutume champions ethically sourced, single-origin beans, meticulously roasted in-house. Their baristas are passionate artisans, expertly crafting each cup using traditional and innovative brewing methods. Whether you're a pour-over aficionado or a cappuccino connoisseur, Coutume's dedication to quality will leave you speechless.

**More than just a Brew:** The minimalist, industrial-chic interior serves as a canvas for artistic expression, featuring rotating exhibitions and installations.  Coupled with an inviting outdoor terrace, Coutume Café is the perfect spot to fuel your Parisian adventures.

### 2. Le Café Kitsuné: Chic, Curated, and Cool

Step into Le Café Kitsuné and be transported to a world of effortless cool. With locations across Paris, this café chain embodies the Parisian spirit of style and sophistication. Their sleek, minimalist design, inspired by Japanese aesthetics, creates a serene and inviting atmosphere. Le Café Kitsuné is renowned for its excellent coffee, carefully selected from independent roasters, and its delightful pastries, many featuring traditional Japanese flavors.

**Beyond the Buzz:** Le Café Kitsuné is a cultural hub. The café frequently hosts DJ sets, live music performances, and art exhibitions, creating a vibrant and eclectic atmosphere.

### 3. Ten Belles: Industrial Chic Meets Specialty Coffee

Ten Belles is a haven for coffee lovers who appreciate both quality and a stylish setting. This industrial-chic café boasts exposed brick walls, vintage furniture, and an open kitchen where you can witness the coffee-making magic firsthand. Ten Belles sources its beans from around the world, highlighting unique flavors and origins. Their baristas are experts in their craft, crafting delicious lattes, cappuccinos, and specialty coffee drinks using innovative brewing methods.

**More than just Beans:** Ten Belles offers a tempting brunch menu featuring all-day breakfast classics, avocado toast, and delectable pastries. The cafe's commitment to sustainability is evident in their use of locally sourced ingredients and compostable packaging.

### 4. Le Pain Quotidien: A Taste of Organic Goodness

Le Pain Quotidien, meaning "The Daily Bread," is a beloved Parisian institution known for its commitment to organic and wholesome ingredients.  This café chain boasts multiple locations across the city, each with a warm and inviting atmosphere.  Their signature sourdough bread, baked fresh daily, is a testament to their dedication to quality. From hearty soups and salads to delicious pastries, Le Pain Quotidien offers a menu that nourishes both body and soul.

**Beyond the Bread:** Le Pain Quotidien fosters a sense of community with its communal tables and warm, welcoming ambiance.  It's the perfect spot to work, socialize, or simply enjoy a leisurely meal.

### 5.  Shakespeare and Company: A Literary Sanctuary

For bookworms and literary enthusiasts, Shakespeare and Company is more than just a café; it's a pilgrimage. This iconic bookstore, located on the Left Bank, has been a haven for writers and readers for over a century.  The café, nestled within the bookstore, offers a cozy and intimate setting to enjoy a cup of coffee or tea while surrounded by the scent of old books. This is the perfect place to lose yourself in a good read, soak up the bohemian atmosphere, and perhaps even spot a fellow writer or literary luminary.

**Beyond the Books:** Shakespeare and Company hosts regular readings, poetry slams, and literary events, making it a vibrant hub for the city's literary scene.


**Your Parisian Café Adventure Awaits!**

These five cafés offer just a glimpse into the rich tapestry of Parisian café culture. Whether you're seeking a caffeine fix, a leisurely brunch, or a literary escape, Paris has a café to suit every taste and mood. So grab a croissant, settle into a comfy armchair, and let the magic of Parisian café culture enchant you.